In [ ]:
# Cài đặt thư viện cần thiết
# pip install transformers
# pip install sentence-transformers
# pip install allennlp allennlp-models

from transformers import pipeline, AutoTokenizer, AutoModelForTokenClassification
from sentence_transformers import SentenceTransformer
from allennlp.predictors.predictor import Predictor
import allennlp_models.coref
import torch
from sklearn.metrics.pairwise import cosine_similarity

# Initialize the food extraction pipeline using DistilBERT
tokenizer = AutoTokenizer.from_pretrained("distilbert-for-food-extraction")
model = AutoModelForTokenClassification.from_pretrained("distilbert-for-food-extraction")
food_extraction_pipeline = pipeline("token-classification", model=model, tokenizer=tokenizer)

# Example text
text = """1. Add the quinoa, vegetables, turmeric, and 6 cups of the water to a large pot. 
Bring to a boil, then lower the heat, cover with a lid, and simmer for 30 minutes.

2. While the vegetables are cooking, add the remaining ⅓ cup of water, chilis, cumin seeds, 
asafoetida, if using, and fenugreek seeds to a large pan. 
Sauté for a few minutes, then add the tomatoes, coriander, cayenne, and garam masala. 
Simmer for about 5 minutes."""


# Perform food extraction (token classification)
food_entities = food_extraction_pipeline(text)

# Co-reference resolution using AllenNLP to handle "mixture"
predictor = Predictor.from_path("https://storage.googleapis.com/allennlp-public-models/coref-spanbert-large-2020.02.27.tar.gz")
coref_result = predictor.predict(document=text)
coref_clusters = coref_result["clusters"]

# Display results of co-reference resolution
print("Co-reference clusters:")
for cluster in coref_clusters:
    print([text[start:end+1] for start, end in cluster])

# Embedding the text using SentenceTransformer
embedding_model = SentenceTransformer('distilbert-base-nli-mean-tokens')
step_embedding = embedding_model.encode(text)

# Example ingredient embedding
ingredient_text = "quinoa, vegetables, water"
ingredient_embedding = embedding_model.encode(ingredient_text)

# Calculate similarity between ingredient and step
similarity = cosine_similarity([ingredient_embedding], [step_embedding])
print(f"Similarity between ingredients and step: {similarity[0][0]}")

# Relation Extraction (dummy example, this can be expanded based on specific models)
# This step could be further expanded with a specific relation extraction model
